In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'n2o': 3.2e-07, 'ch4': 3.6e-06}
band = [9]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0]
  ng_refs = [7]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.8)]
  wgt = [(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9)]
o n2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [2, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.8, 1.66), (1.6, 1.8)]
  wgt = [(0.6, 0.5), (0.6, 0.9)]
o ch4 band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 1.8e-06
 

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.139863                    1 -0.044491
0.000750        2 -0.129924                    2 -0.038324
0.001052        3 -0.107984                    3 -0.024371
0.001476        4 -0.083705                    4 -0.009054
0.002070        5 -0.057227                    5  0.007888
0.002904        6 -0.028395                    6  0.026484
0.004074        7  0.002867                    7  0.046804
0.005714        8  0.036705                    8  0.069191
0.008015        9  0.073187                    9  0.093647
0.011243       10  0.112367                   10  0.120332
0.015771       11  0.154312                   11  0.149404
0.022122       12  0.198988                   12  0.181018
0.031031       13  0.246107                   13  0.215174
0.043528       14  0.288802                   14  0.247555
0.061057       15  0.319907                   15  0.273844
0.085645       16  0.344312                   16  0.297344
0.120136       17  0.367113                   17  0.321450
0.168516       18  0.388308                   18  0.346175
0.236378       19  0.408160                   19  0.371276
0.331549       20  0.427943                   20  0.396910
0.465100       21  0.446748                   21  0.419839
0.652400       22  0.399930                   22  0.392269
0.915100       23  0.280106                   23  0.303923
1.283650       24  0.172182                   24  0.211685
1.800600       25  0.099982                   25  0.136241
2.525700       26  0.054701                   26  0.077738
3.542800       27  0.027291                   27  0.036038
4.969550       28  0.014873                   28  0.013832
6.970850       29  0.012739                   29  0.008343
9.778100       30  0.012288                   30  0.008555
13.715850      31  0.010522                   31  0.008332
19.239350      32  0.007630                   32  0.006740
26.987250      33  0.006908                   33  0.006873
37.855300      34  0.008742                   34  0.008995
53.100050      35  0.009884                   35  0.010334
73.887500      36  0.010290                   36  0.010855
97.662500      37  0.009998                   37  0.010426
121.437500     38  0.007955                   38  0.007980
145.212500     39  0.005058                   39  0.004342
168.987500     40  0.002280                   40  0.000580
192.762500     41 -0.000661                   41 -0.003360
216.537500     42 -0.003992                   42 -0.007622
240.312500     43 -0.007911                   43 -0.012329
264.087500     44 -0.012905                   44 -0.017886
287.862500     45 -0.019319                   45 -0.024232
311.637500     46 -0.022436                   46 -0.026938
335.412500     47 -0.018960                   47 -0.023265
359.187500     48 -0.013168                   48 -0.016973
382.962500     49 -0.007403                   49 -0.010580
406.737500     50 -0.001133                   50 -0.003815
430.512500     51  0.005480                   51  0.003236
454.287500     52  0.012373                   52  0.010227
478.062500     53  0.019451                   53  0.017334
501.837500     54  0.026640                   54  0.024661
525.612500     55  0.033853                   55  0.032465
549.387500     56  0.040951                   56  0.040302
573.162500     57  0.047775                   57  0.047611
596.937500     58  0.054103                   58  0.054144
620.712500     59  0.059723                   59  0.059749
644.487500     60  0.064466                   60  0.065117
668.262500     61  0.068260                   61  0.069707
692.037500     62  0.071163                   62  0.073679
715.812500     63  0.073357                   63  0.077021
739.587500     64  0.075160                   64  0.079869
763.362500     65  0.076988                   65  0.082518
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -6.553508  0.000000 -6.553508 -6.517929  9.404495e-08   
0.000624    2     -6.553525  0.000007 -6.553518 -6.517937  4.430019e-06   
0.000876    3     -6.553532  0.000010 -6.553522 -6.517940  6.277579e-06   
0.001229    4     -6.553541  0.000014 -6.553527 -6.517944  9.166441e-06   
0.001723    5     -6.553553  0.000021 -6.553532 -6.517949  1.367336e-05   
0.002417    6     -6.553568  0.000032 -6.553536 -6.517955  2.069953e-05   
0.003391    7     -6.553588  0.000049 -6.553540 -6.517963  3.164051e-05   
0.004757    8     -6.553614  0.000075 -6.553539 -6.517972  4.864784e-05   
0.006672    9     -6.553646  0.000115 -6.553531 -6.517983  7.506345e-05   
0.009359    10    -6.553685  0.000177 -6.553508 -6.517994  1.160344e-04   
0.013128    11    -6.553730  0.000273 -6.553457 -6.518004  1.794885e-04   
0.018415    12    -6.553781  0.000420 -6.553361 -6.518009  2.776131e-04   
0.025830    13    -6.553831  0.000645 -6.553186 -6.518001  4.290962e-04   
0.036232    14    -6.553869  0.000986 -6.552883 -6.517969  6.624235e-04   
0.050823    15    -6.553881  0.001498 -6.552383 -6.517896  1.017075e-03   
0.071291    16    -6.553851  0.002243 -6.551608 -6.517760  1.544843e-03   
0.100000    17    -6.553749  0.003312 -6.550437 -6.517525  2.321541e-03   
0.140271    18    -6.553516  0.004831 -6.548686 -6.517131  3.460783e-03   
0.196760    19    -6.553055  0.006968 -6.546087 -6.516478  5.124928e-03   
0.275997    20    -6.552196  0.009940 -6.542256 -6.515410  7.541624e-03   
0.387100    21    -6.550636  0.014012 -6.536624 -6.513668  1.102407e-02   
0.543100    22    -6.547841  0.019472 -6.528369 -6.510854  1.596907e-02   
0.761700    23    -6.543784  0.025772 -6.518013 -6.506964  2.223819e-02   
1.068500    24    -6.539296  0.031463 -6.507833 -6.502612  2.893214e-02   
1.498800    25    -6.534838  0.035782 -6.499056 -6.498162  3.527317e-02   
2.102400    26    -6.530536  0.038629 -6.491907 -6.493762  4.061571e-02   
2.949000    27    -6.526573  0.040151 -6.486422 -6.489639  4.429026e-02   
4.136600    28    -6.523317  0.040735 -6.482582 -6.486180  4.590109e-02   
5.802500    29    -6.520959  0.041312 -6.479647 -6.483589  4.604058e-02   
8.139200    30    -6.519226  0.043105 -6.476121 -6.481654  4.641455e-02   
11.417000   31    -6.518130  0.046780 -6.471350 -6.480459  4.854200e-02   
16.014700   32    -6.518390  0.052770 -6.465620 -6.480838  5.345936e-02   
22.464000   33    -6.521715  0.061924 -6.459791 -6.484571  6.234141e-02   
31.510500   34    -6.529389  0.077000 -6.452389 -6.493047  7.818350e-02   
44.200100   35    -6.541467  0.102219 -6.439248 -6.506705  1.053640e-01   
62.000000   36    -6.559380  0.140972 -6.418408 -6.527604  1.480549e-01   
85.775000   37    -6.584333  0.194905 -6.389428 -6.557714  2.087391e-01   
109.550000  38    -6.610362  0.249093 -6.361270 -6.590042  2.704319e-01   
133.325000  39    -6.639348  0.300483 -6.338865 -6.626395  3.292625e-01   
157.100000  40    -6.673644  0.349023 -6.324621 -6.668919  3.840158e-01   
180.875000  41    -6.713935  0.395737 -6.318199 -6.718150  4.348799e-01   
204.650000  42    -6.761100  0.441039 -6.320061 -6.774923  4.821896e-01   
228.425000  43    -6.816213  0.484909 -6.331304 -6.840474  5.262725e-01   
252.200000  44    -6.881050  0.527465 -6.353585 -6.916638  5.677117e-01   
275.975000  45    -6.958863  0.568934 -6.389928 -7.006556  6.072520e-01   
299.750000  46    -7.054970  0.610632 -6.444337 -7.114101  6.465441e-01   
323.525000  47    -7.168341  0.660817 -6.507524 -7.236677  6.932480e-01   
347.300000  48    -7.290169  0.729248 -6.560922 -7.365940  7.569813e-01   
371.075000  49    -7.414845  0.816837 -6.598007 -7.496665  8.398999e-01   
394.850000  50    -7.543726  0.924870 -6.618856 -7.630021  9.434553e-01   
418.625000  51    -7.678024  1.055976 -6.622048 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')